In [146]:
import os
import pandas as pd
import numpy as np

# Replace non-numeric values like 'Not Provided' with NaN and then specify column types
def load_salary_data(file_path):
    # Load the CSV with low_memory=False
    salary_data = pd.read_csv(file_path, low_memory=False)
    
    # Replace 'Not Provided' or other non-numeric values with NaN
    salary_data.replace('Not Provided', np.nan, inplace=True)
    
    # Convert columns to numeric, ignoring errors for any problematic values
    salary_data['BasePay'] = pd.to_numeric(salary_data['BasePay'], errors='coerce')
    salary_data['OvertimePay'] = pd.to_numeric(salary_data['OvertimePay'], errors='coerce')
    salary_data['OtherPay'] = pd.to_numeric(salary_data['OtherPay'], errors='coerce')
    salary_data['Benefits'] = pd.to_numeric(salary_data['Benefits'], errors='coerce')

    return salary_data

# Load the salary data
file_path = "C:/Users/HP/OneDrive/Documents/SalaryData.csv"
salary_data = load_salary_data(file_path)

# Display the first few rows to verify the data
salary_data.head()


EmployeeName                                        JobTitle  \
0     NATHANIEL FORD  GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY   
1       GARY JIMENEZ                 CAPTAIN III (POLICE DEPARTMENT)   
2     ALBERT PARDINI                 CAPTAIN III (POLICE DEPARTMENT)   
3  CHRISTOPHER CHONG            WIRE ROPE CABLE MAINTENANCE MECHANIC   
4    PATRICK GARDNER    DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)   

     BasePay  OvertimePay   OtherPay  Benefits   TotalPay  TotalPayBenefits  \
0  167411.18         0.00  400184.25       NaN  567595.43         567595.43   
1  155966.02    245131.88  137811.38       NaN  538909.28         538909.28   
2  212739.13    106088.18   16452.60       NaN  335279.91         335279.91   
3   77916.00     56120.71  198306.90       NaN  332343.61         332343.61   
4  134401.60      9737.00  182234.59       NaN  326373.19         326373.19   

   Year  
0  2011  
1  2011  
2  2011  
3  2011  
4  2011

In [147]:
# Function to get employee details by name
def get_employee_details(name, data):
    try:
        # Search for employee name in the dataset
        employee = data[data['EmployeeName'].str.contains(name, case=False, na=False)]
        
        if employee.empty:
            raise ValueError("Employee not found.")
        
        return employee.to_dict(orient='records')
    
    except ValueError as e:
        return str(e)

# Test the function with an example employee name
get_employee_details("NATHANIEL FORD", salary_data)




[{'EmployeeName': 'NATHANIEL FORD',
  'JobTitle': 'GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY',
  'BasePay': 167411.18,
  'OvertimePay': 0.0,
  'OtherPay': 400184.25,
  'Benefits': nan,
  'TotalPay': 567595.43,
  'TotalPayBenefits': 567595.43,
  'Year': 2011}]

In [148]:
# Load the salary data
salary_data = pd.read_csv("C:/Users/HP/OneDrive/Documents/SalaryData.csv", low_memory=False)

# Fix column name duplication by appending a suffix to duplicate column names
salary_data.columns = pd.Series(salary_data.columns).apply(lambda x, y=salary_data.columns.duplicated(): f"{x}_dup" if y.any() else x)

# Handle duplicate employee names by adding a numeric suffix to duplicates
salary_data['EmployeeName'] = salary_data.groupby('EmployeeName').cumcount().astype(str).radd(salary_data['EmployeeName'] + '_')

# Convert salary data into a dictionary
def salary_data_to_dict(data):
    # Create a dictionary where EmployeeName is the key, and the rest of the columns are the values
    salary_dict = data.set_index('EmployeeName').to_dict(orient='index')
    return salary_dict

# Convert the salary data to a dictionary
salary_dict = salary_data_to_dict(salary_data)

# To avoid exceeding Jupyter's data rate limit, print only part of the dictionary (e.g., first 5 employees)
for key, value in list(salary_dict.items())[:1]:
    print(f"{key}: {value}")


NATHANIEL FORD_0: {'JobTitle': 'GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY', 'BasePay': '167411.18', 'OvertimePay': '0', 'OtherPay': '400184.25', 'Benefits': 'Not Provided', 'TotalPay': 567595.43, 'TotalPayBenefits': 567595.43, 'Year': 2011}


In [153]:
# Load the salary data with error handling for file-related issues
def load_salary_data(file_path):
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File '{file_path}' not found.")
        
        # Try to load the CSV file
        salary_data = pd.read_csv(file_path, low_memory=False)
        return salary_data
    
    except FileNotFoundError as fnf_error:
        print(fnf_error)
        return None
    
    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")
        return None

    except pd.errors.ParserError:
        print("Error: The file contains parsing errors.")
        return None

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Convert salary data into a dictionary with error handling for missing employee names
def salary_data_to_dict(data):
    try:
        # Check if EmployeeName column exists
        if 'EmployeeName' not in data.columns:
            raise KeyError("'EmployeeName' column is missing from the data.")
        
        # Check for duplicate employee names and handle them
        if data['EmployeeName'].duplicated().any():
            print("Warning: Duplicate employee names found. Adding suffixes to ensure uniqueness.")
            data['EmployeeName'] = data.groupby('EmployeeName').cumcount().astype(str).radd(data['EmployeeName'] + '_')
        
        # Convert to dictionary
        salary_dict = data.set_index('EmployeeName').to_dict(orient='index')
        return salary_dict

    except KeyError as key_error:
        print(key_error)
        return None
    
    except Exception as e:
        print(f"An error occurred while processing the data: {e}")
        return None

# Function to get employee details with error handling
def get_employee_details(employee_name, salary_dict):
    try:
        # Check if employee exists in the dictionary
        if employee_name not in salary_dict:
            raise ValueError(f"Employee '{employee_name}' not found in the salary data.")
        
        # Return employee details
        return salary_dict[employee_name]
    
    except ValueError as ve:
        print(ve)
        return None

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


#usage
file_path = "C:/Users/HP/OneDrive/Documents/SalaryData.csv"

# Load the data
salary_data = load_salary_data(file_path)

if salary_data is not None:
    # Convert to dictionary
    salary_dict = salary_data_to_dict(salary_data)
    


In [ ]:
import csv
import zipfile

# Employee data
employee_data = {
    'Name': 'NATHANIEL FORD',
    'JobTitle': 'GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY',
    'BasePay': '167411.18',
    'OvertimePay': '0',
    'OtherPay': '400184.25',
    'Benefits': 'Not Provided',
    'TotalPay': 567595.43,
    'TotalPayBenefits': 567595.43,
    'Year': 2011
}

def export_employee_to_csv_and_zip(employee, folder_name='Employee Profile'):
    # Ensure the directory exists
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # CSV file path
    csv_filename = os.path.join(folder_name, 'employee_details.csv')

    # Write employee details to CSV
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Writing the header
        writer.writerow(['Name', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay', 'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year'])
        # Writing the employee data
        writer.writerow([employee['Name'], employee['JobTitle'], employee['BasePay'], employee['OvertimePay'], employee['OtherPay'], employee['Benefits'], employee['TotalPay'], employee['TotalPayBenefits'], employee['Year']])

    # Zip the CSV file
    zip_filename = os.path.join(folder_name, 'Employee Profile.zip')
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        zipf.write(csv_filename, os.path.basename(csv_filename))
    
    # Optionally remove the original CSV file to leave only the zip
    os.remove(csv_filename)

# Export the employee data to CSV and zip it
export_employee_to_csv_and_zip(employee_data)

